##### Copyright 2021 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

Nota: Puede ejecutar este ejemplo ahora mismo en un bloc de notas estilo Jupyter, ¡no es necesario configurarlo! Simplemente haga clic en "Ejecutar en Google Colab"

<div class="devsite-table-wrapper"><table class="tfo-notebook-buttons" align="left">
<td><a target="_blank" href="https://www.tensorflow.org/tfx/tutorials/model_analysis/tfma_basic"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver en TensorFlow.org</a></td>
<td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/es-419/tfx/tutorials/model_analysis/tfma_basic.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Ejecutar en Google Colab</a></td>
<td><a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/es-419/tfx/tutorials/model_analysis/tfma_basic.ipynb"><img width="32px" src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver fuente en GitHub</a></td>
<td><a target="_blank" href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/es-419/tfx/tutorials/model_analysis/tfma_basic.ipynb"><img width="32px" src="https://www.tensorflow.org/images/download_logo_32px.png">Descargar el bloc de notas</a></td>
</table></div>

# TensorFlow Model Analysis

***Un ejemplo de un componente clave de TensorFlow Extended (TFX)***

[TensorFlow Model Analysis (TFMA)](https://www.tensorflow.org/tfx/guide/tfma) es una biblioteca para evaluar modelos en diferentes segmentos de datos. TFMA ejecuta los cálculos de forma distribuida sobre grandes cantidades de datos con ayuda de [Apache Beam](https://beam.apache.org/documentation/programming-guide/).

Este bloc de notas de Colab de ejemplo ilustra cómo se puede usar TFMA para investigar y visualizar el rendimiento de un modelo con respecto a las características del conjunto de datos. Usaremos un modelo que entrenamos anteriormente y ahora podrá manipular los resultados. El modelo que entrenamos fue para el [ejemplo de taxi de Chicago](https://github.com/tensorflow/tfx/tree/master/tfx/examples/chicago_taxi_pipeline), que emplea el [conjunto de datos de Taxi Trips](https://data.cityofchicago.org/Transportation/Taxi-Trips/wrvz-psew) publicado por la ciudad de Chicago. Explore el conjunto de datos completo en la [interfaz de usuario de BigQuery](https://bigquery.cloud.google.com/dataset/bigquery-public-data:chicago_taxi_trips).

Como modelador y desarrollador, piense en cómo se usan estos datos y los posibles beneficios y daños que pueden causar las predicciones de un modelo. Un modelo como este podría reforzar los prejuicios y las disparidades sociales. ¿Es una característica relevante para el problema que desea resolver o introducirá un sesgo? Para obtener más información, lea sobre <a target="_blank" href="https://developers.google.com/machine-learning/fairness-overview/">la equidad del aprendizaje automático</a>.

Nota: Para entender TFMA y cómo funciona con Apache Beam, tendrá que saber un poco sobre Apache Beam. La <a target="_blank" href="https://beam.apache.org/documentation/programming-guide/">Guía de programación de Beam</a> es un excelente punto de partida.

Las columnas del conjunto de datos son las siguientes:

<table>
<tr>
<td>pickup_community_area</td>
<td>fare</td>
<td>trip_start_month</td>
</tr>
<tr>
<td>trip_start_hour</td>
<td>trip_start_day</td>
<td>trip_start_timestamp</td>
</tr>
<tr>
<td>pickup_latitude</td>
<td>pickup_longitude</td>
<td>dropoff_latitude</td>
</tr>
<tr>
<td>dropoff_longitude</td>
<td>trip_miles</td>
<td>pickup_census_tract</td>
</tr>
<tr>
<td>dropoff_census_tract</td>
<td>payment_type</td>
<td>company</td>
</tr>
<tr>
<td>trip_seconds</td>
<td>dropoff_community_area</td>
<td>tips</td>
</tr>
</table>

## Cómo instalar extensiones de Jupyter

Nota: Si se ejecutan en un bloc de notas Jupyter local, estas extensiones de Jupyter se deben instalar en el entorno antes de ejecutar Jupyter.

```bash
jupyter nbextension enable --py widgetsnbextension --sys-prefix
jupyter nbextension install --py --symlink tensorflow_model_analysis --sys-prefix
jupyter nbextension enable --py tensorflow_model_analysis --sys-prefix
```

## Cómo instalar TensorFlow Model Analysis (TFMA)

Esto incorporará todas las dependencias y solo demora un minuto.


In [ ]:
# Upgrade pip to the latest, and install TFMA.
!pip install -U pip
!pip install tensorflow-model-analysis

**Ahora debe reiniciar el tiempo de ejecución antes de ejecutar las celdas que se muestran a continuación.**

In [ ]:
# This setup was tested with TF 2.10 and TFMA 0.41 (using colab), but it should
# also work with the latest release.
import sys

# Confirm that we're using Python 3
assert sys.version_info.major==3, 'This notebook must be run using Python 3.'

import tensorflow as tf
print('TF version: {}'.format(tf.__version__))
import apache_beam as beam
print('Beam version: {}'.format(beam.__version__))
import tensorflow_model_analysis as tfma
print('TFMA version: {}'.format(tfma.__version__))

**NOTA: Para continuar, el resultado anterior no puede contener errores. Vuelva a ejecutar la instalación si aún ve errores. Además, asegúrese de reiniciar el tiempo de ejecución/kernel antes de pasar al siguiente paso.**

## Cómo cargar los archivos

Descargaremos un archivo tar que contiene todo lo que necesitamos. Eso incluye lo siguiente:

- Conjuntos de datos de entrenamiento y evaluación.
- Esquema de datos
- Entrenamiento y servicio de modelos guardados (keras y estimator) y modelos guardados de evaluación (estimator).

In [ ]:
# Download the tar file from GCP and extract it
import io, os, tempfile
TAR_NAME = 'saved_models-2.2'
BASE_DIR = tempfile.mkdtemp()
DATA_DIR = os.path.join(BASE_DIR, TAR_NAME, 'data')
MODELS_DIR = os.path.join(BASE_DIR, TAR_NAME, 'models')
SCHEMA = os.path.join(BASE_DIR, TAR_NAME, 'schema.pbtxt')
OUTPUT_DIR = os.path.join(BASE_DIR, 'output')

!curl -O https://storage.googleapis.com/artifacts.tfx-oss-public.appspot.com/datasets/{TAR_NAME}.tar
!tar xf {TAR_NAME}.tar
!mv {TAR_NAME} {BASE_DIR}
!rm {TAR_NAME}.tar

print("Here's what we downloaded:")
!ls -R {BASE_DIR}

## Cómo parsear el esquema

Entre las cosas que descargamos, había un esquema para nuestros datos creado por [TensorFlow Data Validation](https://www.tensorflow.org/tfx/data_validation/get_started/). Analicémoslo ahora para poder usarlo con TFMA.

In [ ]:
import tensorflow as tf
from google.protobuf import text_format
from tensorflow.python.lib.io import file_io
from tensorflow_metadata.proto.v0 import schema_pb2
from tensorflow.core.example import example_pb2

schema = schema_pb2.Schema()
contents = file_io.read_file_to_string(SCHEMA)
schema = text_format.Parse(contents, schema)

## Cómo usar el esquema para crear TFRecords

Necesitamos darle acceso a TFMA a nuestro conjunto de datos, para eso, tenemos que crear un archivo TFRecords. Podemos usar nuestro esquema para crearlo, ya que nos aporta el tipo correcto para cada característica.

In [ ]:
import csv

datafile = os.path.join(DATA_DIR, 'eval', 'data.csv')
reader = csv.DictReader(open(datafile, 'r'))
examples = []
for line in reader:
  example = example_pb2.Example()
  for feature in schema.feature:
    key = feature.name
    if feature.type == schema_pb2.FLOAT:
      example.features.feature[key].float_list.value[:] = (
          [float(line[key])] if len(line[key]) > 0 else [])
    elif feature.type == schema_pb2.INT:
      example.features.feature[key].int64_list.value[:] = (
          [int(line[key])] if len(line[key]) > 0 else [])
    elif feature.type == schema_pb2.BYTES:
      example.features.feature[key].bytes_list.value[:] = (
          [line[key].encode('utf8')] if len(line[key]) > 0 else [])
  # Add a new column 'big_tipper' that indicates if tips was > 20% of the fare. 
  # TODO(b/157064428): Remove after label transformation is supported for Keras.
  big_tipper = float(line['tips']) > float(line['fare']) * 0.2
  example.features.feature['big_tipper'].float_list.value[:] = [big_tipper]
  examples.append(example)

tfrecord_file = os.path.join(BASE_DIR, 'train_data.rio')
with tf.io.TFRecordWriter(tfrecord_file) as writer:
  for example in examples:
    writer.write(example.SerializeToString())

!ls {tfrecord_file}

## Cómo configurar y ejecutar TFMA

TFMA es compatible con distintos tipos de modelos, incluidos modelos de TF keras, modelos basados ​​en API de firma TF2 genéricas y modelos basados ​​en TF estimator. La guía [get_started](https://www.tensorflow.org/tfx/model_analysis/get_started) tiene la lista completa de los tipos de modelos admitidos y sus restricciones. Para este ejemplo, mostraremos cómo configurar un modelo basado en keras, así como un modelo basado en un estimador que se guardó como [`EvalSavedModel`](https://www.tensorflow.org/tfx/model_analysis/eval_saved_model). Consulte las [preguntas frecuentes](https://www.tensorflow.org/tfx/model_analysis/faq) para ver ejemplos de otras configuraciones.

TFMA ofrece compatibilidad para calcular las métricas que se usaron en el momento del entrenamiento (es decir, las métricas integradas), así como las métricas definidas después de guardar el modelo como parte de los ajustes de configuración de TFMA. Para nuestra [configuración](https://www.tensorflow.org/tfx/model_analysis/setup) de keras, demostraremos cómo agregar nuestras métricas y gráficos manualmente como parte de nuestra configuración (consulte la guía de [métricas](https://www.tensorflow.org/tfx/model_analysis/metrics) para obtener información sobre las métricas y gráficos compatibles). Para la configuración del estimador usaremos las métricas integradas que se guardaron con el modelo. Nuestras configuraciones también incluyen una serie de especificaciones de segmentación que se analizan con más detalle en las siguientes secciones.

Después de crear [`tfma.EvalConfig`](https://www.tensorflow.org/tfx/model_analysis/api_docs/python/tfma/EvalConfig) y [`tfma.EvalSharedModel`](https://www.tensorflow.org/tfx/model_analysis/api_docs/python/tfma/EvalSharedModel) podemos ejecutar TFMA con ayuda de [`tfma.run_model_analysis`](https://www.tensorflow.org/tfx/model_analysis/api_docs/python/tfma/run_model_analysis). Esto creará un [`tfma.EvalResult`](https://www.tensorflow.org/tfx/model_analysis/api_docs/python/tfma/EvalResult) que podremos usar más adelante para representar nuestras métricas y gráficos.

### Keras

In [ ]:
import tensorflow_model_analysis as tfma

# Setup tfma.EvalConfig settings
keras_eval_config = text_format.Parse("""
  ## Model information
  model_specs {
    # For keras (and serving models) we need to add a `label_key`.
    label_key: "big_tipper"
  }

  ## Post training metric information. These will be merged with any built-in
  ## metrics from training.
  metrics_specs {
    metrics { class_name: "ExampleCount" }
    metrics { class_name: "AUC" }
    metrics { class_name: "Precision" }
    metrics { class_name: "Recall" }
    metrics { class_name: "MeanPrediction" }
    metrics { class_name: "Calibration" }
    metrics { class_name: "CalibrationPlot" }
    metrics { class_name: "ConfusionMatrixPlot" }
    # ... add additional metrics and plots ...
  }

  ## Slicing information
  slicing_specs {}  # overall slice
  slicing_specs {
    feature_keys: ["trip_start_hour"]
  }
  slicing_specs {
    feature_keys: ["trip_start_day"]
  }
  slicing_specs {
    feature_values: {
      key: "trip_start_month"
      value: "1"
    }
  }
""", tfma.EvalConfig())

# Create a tfma.EvalSharedModel that points at our keras model.
keras_model_path = os.path.join(MODELS_DIR, 'keras', '2')
keras_eval_shared_model = tfma.default_eval_shared_model(
    eval_saved_model_path=keras_model_path,
    eval_config=keras_eval_config)

keras_output_path = os.path.join(OUTPUT_DIR, 'keras')

# Run TFMA
keras_eval_result = tfma.run_model_analysis(
    eval_shared_model=keras_eval_shared_model,
    eval_config=keras_eval_config,
    data_location=tfrecord_file,
    output_path=keras_output_path)

### Estimator

In [ ]:
import tensorflow_model_analysis as tfma

# Setup tfma.EvalConfig settings
estimator_eval_config = text_format.Parse("""
  ## Model information
  model_specs {
    # To use EvalSavedModel set `signature_name` to "eval".
    signature_name: "eval"
  }

  ## Post training metric information. These will be merged with any built-in
  ## metrics from training.
  metrics_specs {
    metrics { class_name: "ConfusionMatrixPlot" }
    # ... add additional metrics and plots ...
  }

  ## Slicing information
  slicing_specs {}  # overall slice
  slicing_specs {
    feature_keys: ["trip_start_hour"]
  }
  slicing_specs {
    feature_keys: ["trip_start_day"]
  }
  slicing_specs {
    feature_values: {
      key: "trip_start_month"
      value: "1"
    }
  }
""", tfma.EvalConfig())

# Create a tfma.EvalSharedModel that points at our eval saved model.
estimator_base_model_path = os.path.join(
    MODELS_DIR, 'estimator', 'eval_model_dir')
estimator_model_path = os.path.join(
    estimator_base_model_path, os.listdir(estimator_base_model_path)[0])
estimator_eval_shared_model = tfma.default_eval_shared_model(
    eval_saved_model_path=estimator_model_path,
    eval_config=estimator_eval_config)

estimator_output_path = os.path.join(OUTPUT_DIR, 'estimator')

# Run TFMA
estimator_eval_result = tfma.run_model_analysis(
    eval_shared_model=estimator_eval_shared_model,
    eval_config=estimator_eval_config,
    data_location=tfrecord_file,
    output_path=estimator_output_path)

## Cómo visualizar métricas y gráficos

Ahora que ya ejecutamos la evaluación, usemos TFMA para analizar nuestras visualizaciones. Para los siguientes ejemplos, visualizaremos los resultados de la ejecución de la evaluación en el modelo keras. Si desea ver el modelo basado en el estimador, actualice `eval_result_path` para que apunte a nuestra variable `estimator_output_path`.

In [ ]:
eval_result_path = keras_output_path
# eval_result_path = estimator_output_path

eval_result = keras_eval_result
# eval_result = estimator_eval_result

### Cómo representar métricas

TFMA proporciona API de marco de datos en [`tfma.experimental.dataframe`](https://www.tensorflow.org/tfx/model_analysis/api_docs/python/tfma/experimental) para cargar la salida materializada como [`Pandas DataFrames`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html). Para ver métricas, puede usar `metrics_as_dataframes(tfma.load_metrics(eval_path))`, que devuelve un objeto que potencialmente contiene varios DataFrames, uno para cada tipo de valor de métrica (`double_value`, `confusion_matrix_at_thresholds`, `bytes_value` y `array_value`). Los DataFrames específicos que se completan dependen del resultado de la evaluación. Aquí, mostramos el DataFrame `double_value` a modo de ejemplo.

In [ ]:
import tensorflow_model_analysis.experimental.dataframe as tfma_dataframe
dfs = tfma_dataframe.metrics_as_dataframes(
  tfma.load_metrics(eval_result_path))

display(dfs.double_value.head())

Cada uno de los DataFrames tiene un índice múltiple de columnas con las columnas de nivel superior: `slices` , `metric_keys` y `metric_values` . Las columnas exactas de cada grupo pueden cambiar en función de la carga útil. Podemos usar la API `DataFrame.columns` para inspeccionar todas las columnas de índices múltiples. Por ejemplo, las columnas de segmentos son 'Overall', 'trip_start_day', 'trip_start_hour' y 'trip_start_month', que se configuran mediante `slicing_specs` en `eval_config`.

In [ ]:
print(dfs.double_value.columns)

### Pivote automático

El DataFrame tiene un diseño detallado para que no se pierda información de la carga útil. Sin embargo, a veces, para el consumo directo, es posible que deseemos organizar la información de una forma más concisa, pero con pérdidas: segmentos como filas y métricas como columnas. TFMA ofrece la API `auto_pivot` con esta finalidad. La utilidad pivota sobre todas las columnas no únicas dentro de `metric_keys` y condensa todos los segmentos en una columna `stringified_slices` de forma predeterminada.

In [ ]:
tfma_dataframe.auto_pivot(dfs.double_value).head()

### Cómo filtrar segmentos

Dado que las salidas son DataFrames, se puede usar cualquier API nativa de DataFrame para segmentar y fragmentar el DataFrame. Por ejemplo, si solo estamos interesados ​​en `trip_start_hour` de 1, 3, 5, 7 y no en `trip_start_day`, podemos usar la lógica de filtrado `.loc` de DataFrame. Nuevamente, usamos la función `auto_pivot` para reorganizar el DataFrame en la vista comparativa entre segmentos y métricas después de ejecutar el filtrado.

In [ ]:
df_double = dfs.double_value
df_filtered = (df_double
  .loc[df_double.slices.trip_start_hour.isin([1,3,5,7])]
)
display(tfma_dataframe.auto_pivot(df_filtered))

### Cómo ordenar por valores de métricas

También podemos ordenar los segmentos por valor de métricas. Como ejemplo, mostramos cómo ordenar segmentos en el DataFrame anterior mediante AUC ascendente, para que podamos encontrar segmentos con bajo rendimiento. Esto implica dos pasos: pivotar automáticamente para que los segmentos se representen como filas y las columnas como métricas, y luego ordenar el DataFrame dinamizado por la columna AUC.

In [ ]:
# Pivoted table sorted by AUC in ascending order.
df_sorted = (
    tfma_dataframe.auto_pivot(df_double)
    .sort_values(by='auc', ascending=True)
    )
display(df_sorted.head())

### Cómo representar gráficos

Cualquier gráfico que se haya agregado a `tfma.EvalConfig` como `metric_specs` después al entrenamiento se puede mostrar con [`tfma.view.render_plot`](https://www.tensorflow.org/tfx/model_analysis/api_docs/python/tfma/view/render_plot).

Al igual que con las métricas, los gráficos se pueden ver por segmento. A diferencia de las métricas, solo se pueden mostrar gráficos para un valor de segmento específico, por lo que se debe usar `tfma.SlicingSpec` y se debe especificar tanto el nombre como el valor de la característica de segmento. Si no se especifica ningún segmento, se utilizan los gráficos del segmento `Overall`.

En el siguiente ejemplo, mostramos los gráficos `CalibrationPlot` y `ConfusionMatrixPlot` que se calcularon para el segmento `trip_start_hour:1`.

In [ ]:
tfma.view.render_plot(
    eval_result,
    tfma.SlicingSpec(feature_values={'trip_start_hour': '1'}))

## Cómo hacer un seguimiento del rendimiento del modelo a lo largo del tiempo

Usaremos su conjunto de entrenamiento para entrenar su modelo y, con suerte, será representativo de su conjunto de datos de prueba y de los datos que se enviarán a su modelo en producción. Sin embargo, si bien los datos de las solicitudes de inferencia pueden seguir siendo los mismos que los datos de entrenamiento, en muchos casos comenzarán a cambiar lo suficiente como para que cambie el rendimiento de su modelo.

Eso significa que debe monitorear y medir el desempeño de su modelo de manera continua, para poder estar al tanto de los cambios y reaccionar ante ellos. Veamos cómo puede ayudarnos TFMA.

Carguemos 3 ejecuciones de modelos diferentes y usemos TFMA para ver cómo se comparan usando [`render_time_series`](https://www.tensorflow.org/tfx/model_analysis/api_docs/python/tfma/view/render_time_series).

In [ ]:
# Note this re-uses the EvalConfig from the keras setup.

# Run eval on each saved model
output_paths = []
for i in range(3):
  # Create a tfma.EvalSharedModel that points at our saved model.
  eval_shared_model = tfma.default_eval_shared_model(
      eval_saved_model_path=os.path.join(MODELS_DIR, 'keras', str(i)),
      eval_config=keras_eval_config)

  output_path = os.path.join(OUTPUT_DIR, 'time_series', str(i))
  output_paths.append(output_path)

  # Run TFMA
  tfma.run_model_analysis(eval_shared_model=eval_shared_model,
                          eval_config=keras_eval_config,
                          data_location=tfrecord_file,
                          output_path=output_path)

En primer lugar, imaginaremos que entrenamos e implementamos nuestro modelo ayer y ahora queremos ver cómo funciona con los nuevos datos que llegan hoy. La visualización comenzará mostrando el AUC. Desde la interfaz de usuario puede ejecutar las siguientes acciones:

- Agregar otras métricas con el menú "Agregar serie de métricas"
- Cerrar los grafos no deseados haciendo clic en x
- Pasar el cursor sobre los puntos de datos (los extremos de los segmentos de línea en el grafo) para obtener más detalles

Nota: En los gráficos de series de métricas, el eje X es el nombre del directorio del modelo de ejecución del modelo que está examinando. Estos nombres en sí mismos no tienen significado.

In [ ]:
eval_results_from_disk = tfma.load_eval_results(output_paths[:2])

tfma.view.render_time_series(eval_results_from_disk)

Ahora imaginaremos que ha pasado otro día y queremos ver cómo les va con los nuevos datos que llegan hoy, en contraste con los dos días anteriores:

In [ ]:
eval_results_from_disk = tfma.load_eval_results(output_paths)

tfma.view.render_time_series(eval_results_from_disk)

## Validación de modelos

TFMA se puede configurar para evaluar varios modelos al mismo tiempo. Normalmente, esto se hace para comparar un nuevo modelo con una línea base (como el modelo actualmente en servicio) para determinar cuáles son las diferencias de rendimiento en las métricas (por ejemplo, AUC, etc.) en relación con la línea base. Cuando se configuran [los umbrales](https://www.tensorflow.org/tfx/model_analysis/api_docs/python/tfma/MetricThreshold), TFMA genera un registro [`tfma.ValidationResult`](https://www.tensorflow.org/tfx/model_analysis/api_docs/python/tfma/ValidationResult) que indica si el rendimiento coincide con las expectativas.

Reconfiguremos nuestra evaluación de keras para comparar dos modelos: un candidato y una línea base. También validaremos el desempeño del candidato con respecto a la línea base estableciendo un [`tmfa.MetricThreshold`](https://www.tensorflow.org/tfx/model_analysis/api_docs/python/tfma/MetricThreshold) en la métrica AUC.

In [ ]:
# Setup tfma.EvalConfig setting
eval_config_with_thresholds = text_format.Parse("""
  ## Model information
  model_specs {
    name: "candidate"
    # For keras we need to add a `label_key`.
    label_key: "big_tipper"
  }
  model_specs {
    name: "baseline"
    # For keras we need to add a `label_key`.
    label_key: "big_tipper"
    is_baseline: true
  }

  ## Post training metric information
  metrics_specs {
    metrics { class_name: "ExampleCount" }
    metrics { class_name: "BinaryAccuracy" }
    metrics { class_name: "BinaryCrossentropy" }
    metrics {
      class_name: "AUC"
      threshold {
        # Ensure that AUC is always > 0.9
        value_threshold {
          lower_bound { value: 0.9 }
        }
        # Ensure that AUC does not drop by more than a small epsilon
        # e.g. (candidate - baseline) > -1e-10 or candidate > baseline - 1e-10
        change_threshold {
          direction: HIGHER_IS_BETTER
          absolute { value: -1e-10 }
        }
      }
    }
    metrics { class_name: "AUCPrecisionRecall" }
    metrics { class_name: "Precision" }
    metrics { class_name: "Recall" }
    metrics { class_name: "MeanLabel" }
    metrics { class_name: "MeanPrediction" }
    metrics { class_name: "Calibration" }
    metrics { class_name: "CalibrationPlot" }
    metrics { class_name: "ConfusionMatrixPlot" }
    # ... add additional metrics and plots ...
  }

  ## Slicing information
  slicing_specs {}  # overall slice
  slicing_specs {
    feature_keys: ["trip_start_hour"]
  }
  slicing_specs {
    feature_keys: ["trip_start_day"]
  }
  slicing_specs {
    feature_keys: ["trip_start_month"]
  }
  slicing_specs {
    feature_keys: ["trip_start_hour", "trip_start_day"]
  }
""", tfma.EvalConfig())

# Create tfma.EvalSharedModels that point at our keras models.
candidate_model_path = os.path.join(MODELS_DIR, 'keras', '2')
baseline_model_path = os.path.join(MODELS_DIR, 'keras', '1')
eval_shared_models = [
  tfma.default_eval_shared_model(
      model_name=tfma.CANDIDATE_KEY,
      eval_saved_model_path=candidate_model_path,
      eval_config=eval_config_with_thresholds),
  tfma.default_eval_shared_model(
      model_name=tfma.BASELINE_KEY,
      eval_saved_model_path=baseline_model_path,
      eval_config=eval_config_with_thresholds),
]

validation_output_path = os.path.join(OUTPUT_DIR, 'validation')

# Run TFMA
eval_result_with_validation = tfma.run_model_analysis(
    eval_shared_models,
    eval_config=eval_config_with_thresholds,
    data_location=tfrecord_file,
    output_path=validation_output_path)

Al ejecutar evaluaciones con uno o más modelos con respecto a una línea base, TFMA agrega automáticamente métricas diferenciales para todas las métricas calculadas durante la evaluación. Estas métricas reciben el nombre de la métrica correspondiente, pero con `_diff` añadido al nombre de la métrica.

Echemos un vistazo a las métricas producidas por nuestra ejecución:

In [ ]:
tfma.view.render_time_series(eval_result_with_validation)

Ahora veamos el resultado de nuestras comprobaciones de validación. Para ver los resultados de la validación usamos [`tfma.load_validator_result`](https://www.tensorflow.org/tfx/model_analysis/api_docs/python/tfma/load_validation_result). Para nuestro ejemplo, la validación falla porque el AUC está por debajo del umbral.

In [ ]:
validation_result = tfma.load_validation_result(validation_output_path)
print(validation_result.validation_ok)

# Copyright © 2020 Los autores de TensorFlow.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

Nota: Este sitio ofrece aplicaciones que usan datos que fueron modificados para su uso desde su fuente original, www.cityofchicago.org, el sitio web oficial de la ciudad de Chicago. La ciudad de Chicago no garantiza el contenido, la exactitud, la puntualidad o la integridad de ninguno de los datos que se proporcionan en este sitio. Los datos proporcionados en este sitio están sujetos a cambios en cualquier momento. Se entiende que los datos proporcionados en este sitio se usan bajo su propia responsabilidad.